In [2]:
%run common.ipynb

tokenizer.decode(tokenizer.get_data(third_number=True))

decoder = ['P', 'S', 'E', 'U', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '*', '/', '.', ':', '=', '_']
encoder = {'P': 0, 'S': 1, 'E': 2, 'U': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '+': 14, '-': 15, '*': 16, '/': 17, '.': 18, ':': 19, '=': 20, '_': 21}
length check: True
22
cuda


'S12.93=-25.15+-23.54+61.62E'

In [3]:
'''
构造一个批次的训练数据, 包括 正确回答: choice 和 错误回答: reject
并对每个样本进行填充、构造注意力掩码和标签, 使得模型在训练时只关注答案部分
'''
def get_batch_data():

    def pad(data, split, lens):
        # 做个白板
        input_ids = torch.full(
            (len(data), lens),
            tokenizer.encoder['P'],
            device=device
        )  # -> (batch_size, lens)

        # 往白板里黏贴数据
        for i, d in enumerate(data):
            input_ids[i, :len(d)] = torch.LongTensor(d)

        attention_mask = (input_ids != tokenizer.encoder['P']).long()

        # 计算label
        label = input_ids.clone()
        for l, s in zip(label, split):
            # 将序列前 s 个位置, 通常为问题部, 以及所有填充位置都设为 -100
            l[:s] = -100  # 问题
            l[l == tokenizer.encoder['P']] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

    # 正确的问答
    # 设定为 三数求和
    choice = [tokenizer.get_data(third_number=True) for i in range(64)]

    # 分隔位置
    # 对每个生成的序列，找到等号 '=' 在序列中的位置，并加 1。
    # 这个位置将作为问题和答案的分界线，前面的部分代表问题，后面的部分代表答案
    split = [i.index(tokenizer.encoder['=']) + 1 for i in choice]

    # 错误的回答 简单地定义为空回答就可以了
    reject = [d[:s] for d, s in zip(choice, split)]  # 取每个回答的问题部分
    reject = [i + [tokenizer.encoder['E']] for i in reject]  # 将答案部分填充为 End

    # 求最大长度
    lens = max([len(i) for i in choice])
    # print([x.shape for x in pad(choice, split, lens).values()])
    # print([x.shape for x in pad(reject, split, lens).values()])
    return pad(choice, split, lens), pad(reject, split, lens)


get_batch_data()

({'input_ids': tensor([[ 1, 10,  5,  ...,  0,  0,  0],
          [ 1, 11, 11,  ...,  0,  0,  0],
          [ 1,  5, 13,  ...,  2,  0,  0],
          ...,
          [ 1, 15,  6,  ...,  0,  0,  0],
          [ 1, 13,  7,  ...,  0,  0,  0],
          [ 1, 15, 11,  ...,  0,  0,  0]], device='cuda:0'),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
  'label': tensor([[-100, -100, -100,  ..., -100, -100, -100],
          [-100, -100, -100,  ..., -100, -100, -100],
          [-100, -100, -100,  ...,    2, -100, -100],
          ...,
          [-100, -100, -100,  ..., -100, -100, -100],
          [-100, -100, -100,  ..., -100, -100, -100],
          [-100, -100, -100,  ..., -100, -100, -100]], device='cuda:0')},
 {'input_ids': tensor([[ 1, 10,  5,  ...,  0,  0,  0],
          

为什么在加载 model_dpo_ref 时将其转换到 train 模式？

保持前向传播行为一致： 
模型在训练模式（train mode）和评估模式（eval mode）下的行为可能不同，例如 dropout 和 batch normalization 等层在训练时会有随机性或统计更新。如果参考模型与当前模型的前向传播行为不同，就会导致两者输出的概率分布不一致，从而影响 KL 散度的计算。

与当前模型一致的训练状态：
在 DPO 方法中，参考模型用于为当前模型提供一个固定的基准。虽然参考模型的参数不更新，但在训练过程中，保持两者在相同的模式下可以使得它们在相同训练条件下输出，从而使得对比更准确。

In [4]:
model_dpo = torch.load('gen.model')
model_dpo.to(device)
model_dpo.train()  # 设置为训练模式

model_dpo_ref = torch.load('gen.model')
model_dpo_ref.to(device)
model_dpo_ref.train()  # 设置为训练模式

/tmp/ipykernel_3611/4021970835.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dpo = torch.load('gen.model')


/tmp/ipykernel_3611/4021970835.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dpo_ref = torch.load('gen.model')


ModelGEN(
  (feature): LlamaModel(
    (embed_tokens): Embedding(22, 64)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=64, out_features=64, bias=False)
          (k_proj): Linear(in_features=64, out_features=64, bias=False)
          (v_proj): Linear(in_features=64, out_features=64, bias=False)
          (o_proj): Linear(in_features=64, out_features=64, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=64, out_features=64, bias=False)
          (up_proj): Linear(in_features=64, out_features=64, bias=False)
          (down_proj): Linear(in_features=64, out_features=64, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((64,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((64,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((64,), eps=1e-06)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (fc_out): Linear

联合对数概率是通过对模型每个时间步生成正确 token 的对数概率求和而得到的, 对每个样本来说，它等于:

$$
\log P(y\mid x)=\sum_{t}\log P(y_t\mid y_{<t}, x)
$$

In [5]:
' 计算每个词的概率, 计算模型对两种不同回答 choice 与 reject 的联合生成概率的对数值, 并返回它们之间的差异 '
def get_prob_log(model, choice, reject):
    b = choice['input_ids'].shape[0]  # b=64

    # 合并两部分输入,同时计算以提高效率
    # [2b, 30]
    input_ids = torch.cat([choice['input_ids'], reject['input_ids']], dim=0)
    attention_mask = torch.cat([choice['attention_mask'], reject['attention_mask']], dim=0)
    label = torch.cat([choice['label'], reject['label']], dim=0)

    # [2b, 30, 22]
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    # print(out.shape) -> torch.Size([128, 30, 22])
    # -> (batch_size, seq_len, vocab_size)

    # 偏移以对齐
    # 为了计算生成下一个词的概率, 需要对输出和标签进行时间步的对齐
    label = label[:, 1:]  # [2b, 29]
    out = out[:, :-1]  # [2b, 29, 22]

    # 取所有字的预测概率, 因为要求联合概率, 所以取对数
    # 在 词表 维度做 softmax 得到预测概率分布, 加上一个很小的常数防止 log(0), 然后取对数
    # 这一步得到的是每个时间步、每个 token 的对数概率
    out = (out.softmax(dim=2) + 1e-8).log()

    # 取预测到 label 的概率
    # 索引不能是负数, 所以这里把负数置0
    # 将真实标签扩展一个维度, 然后使用 gather 操作, 从 out 中选出每个时间步对应真实 token 的对数概率
    index = label.clone().unsqueeze(2)  # -> torch.Size([128, 29, 1]) 
    index[index == -100] = 0
    prob = out.gather(2, index=index).squeeze(2)  # -> (2b, seq_len-1): torch.Size([128, 29])
    
    # 只取答案部分的loss, 筛选后, 所有答案的概率对数求和
    prob = (prob * (label != -100)).sum(1)

    # choice 和 reject 的预测概率求差
    # 前 b 是 choice, 后 b 是 reject
    # L_w - L_l  = \log\pi_\theta(y_w|x) - \log\pi_\theta(y_l|x)
    return prob[:b] - prob[b:]

get_prob_log(model_dpo, *get_batch_data())

tensor([-36.3975, -41.2956, -38.5050, -33.2261, -39.9436, -33.6403, -38.7957,
        -33.8615, -35.3346, -36.6786, -33.2887, -41.1539, -36.2859, -33.5000,
        -35.1207, -42.6176, -32.4942, -40.6792, -34.1383, -36.3579, -32.3941,
        -34.6535, -42.4205, -35.9236, -36.9693, -33.9081, -39.1094, -33.1617,
        -38.3072, -35.2617, -33.2931, -32.9153, -34.7415, -34.6575, -37.3955,
        -35.3297, -37.8426, -35.4901, -40.4166, -33.1556, -32.6194, -35.3885,
        -31.9937, -39.4524, -32.2923, -32.0736, -29.5068, -34.6358, -35.5317,
        -37.8613, -34.9341, -35.3608, -33.2361, -35.2435, -40.4090, -35.1232,
        -35.7186, -32.2918, -34.1016, -32.7183, -32.1375, -35.9033, -38.8661,
        -31.4803], device='cuda:0', grad_fn=<SubBackward0>)

$$
\begin{align}
\mathcal{L}_{\text{DPO}}(\pi_{\theta};\pi_{\text{ref}}) &=-\mathbb{E}_{x,y_{w},y_{l}\sim \mathcal{D}}\left[ \log\sigma\left( \beta \log \frac{\pi_{\theta}(y_{w}|x)}{\pi_{\text{ref}}(y_{w}|x)} - \beta \log \frac{\pi_{\theta}(y_{l}|x)}{\pi_{\text{ref}}(y_{l}|x)} \right) \right]  \\
&= -\mathbb{E}_{x,y_{w},y_{l}\sim \mathcal{D}}\left[ \log\sigma \bigg(\beta \right( ( \log\pi (y_w|x) - \log\pi (y_l|x) ) -  ( \log\pi_{\text{ref}}(y_w|x) - \log\pi_{\text{ref}}(y_l|x) ) \left)\bigg) \right] 
\end{align}
$$

In [6]:
optimizer = torch.optim.Adam(
    model_dpo.parameters(),
    lr=1e-4,
    betas=(0.9, 0.999),
    eps=1e-8
)

for i in range(20_0000):
    choice, reject = get_batch_data()

    # 两个模型分别计算概率对数
    prob_log = get_prob_log(model_dpo, choice, reject)  # L_w - L_l  = \log\pi_\theta(y_w|x) - \log\pi_\theta(y_l|x)
    with torch.no_grad():
        prob_log_ref = get_prob_log(model_dpo_ref, choice, reject)  # # L_w^{\text{ref}} - L_l^{\text{ref}}

    # 两份概率计算 KL 散度
    kl = -0.1 * (prob_log - prob_log_ref)

    # 以kl散度计算loss
    loss = (kl.sigmoid() + 1e-8).log().mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 2000 == 0:
        question = tokenizer.get_data(third_number=True)
        question = question[:question.index(tokenizer.encoder['=']) + 1]
        question = torch.LongTensor(question).unsqueeze(0).to(device)

        gen = generate(model_dpo, question)
        print(i, tokenizer.decode(gen[0].tolist()))

model_dpo.to('cpu')
torch.save(model_dpo, 'dpo.model')

0 S38.66=-54.10+63.5/-991.90E
2000 S-0.95=10.74/-8.48+-1.67E
4000 S-784.15=58.70*-13.22+47.97E
6000 S24.08=-54.01/-81.17+22.06E
8000 S3100.93=-50.69*-65.81+-48.18E
10000 S100.26=78.32+69.76+4.49E
12000 S-2464.63=-42.03*55.97+-23.99E
14000 S-113.92=-22.43-79.28+-17.07E
16000 S122.67=77.95--14.93+34.23E
18000 S-22.73=-37.63-6.48+34.90E
20000 S-13.32=-7.20+-42.35+39.57E
22000 S-43.89=-1.13+1.92+-46.45E
24000 S-110.33=-57.24-44.87+-8.08E
26000 S-0.17=34.01/-57.09+-0.96E
28000 S-2.30=-26.17--15.81+12.24E
30000 S-53.90=15.98+-87.02+19.26E
32000 S251.24=-16.03*-17.50+0.25E
34000 S92.52=24.50--15.78+50.16E
36000 S5397.76=58.27*92.93+64.71E
38000 S-1317.96=-70.50*19.17+-3.34E
40000 S4697.70=47.65*88.29+64.41E
42000 S-50.51=-84.35+48.00+-11.67E
44000 S10.44=-98.18/58.12+12.47E
46000 S-534.05=36.95*-15.62+13.51E
48000 S1136.50=16.76*68.07+39.75E
50000 S18.31=56.96+69.55+-99.93E
52000 S130.28=23.66--68.28+35.58E
54000 S-28.05=9.51/88.61+-28.36E
56000 S4954.76=62.31*82.60+-28.61E
58000 S-15.25=-39.